In [ ]:
! pip install rfpimp -q

In [ ]:
pip install imblearn

In [ ]:
pip install minisom

In [ ]:
# https://github.com/JustGlowing/minisom

In [ ]:
pip install tslearn

In [ ]:
# https://tslearn.readthedocs.io/en/stable/index.html

In [ ]:
pip install h5py

In [ ]:
! pip install tslearn
! pip uninstall scikit-learn --yes 
! pip install scikit-learn==0.24.1

In [ ]:
# Native libraries
import os
import math
import warnings

# Essential Libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.ticker as ticker
# Preprocessing
from sklearn.preprocessing import MinMaxScaler

# Algorithms
from minisom import MiniSom
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE, MDS
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram
from tqdm.autonotebook import tqdm
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import statsmodels.api as sm


warnings.filterwarnings("ignore")
sns.set_style("darkgrid")

SEED=23


In [ ]:
noise_test = pd.read_csv('1_cleaned_data.csv')

In [ ]:
# All the columns
noise_test.columns

In [ ]:
# There are a series of complaints not belonging to Westminster
# Select only the records in Westminster and in the sub group of 'Noise Complaint'
lsoanot = ['Brent 032E', 'Brent 034B', 'Camden 017C', 'Camden 017D',
       'Camden 018D', 'Camden 020A', 'Camden 020B', 'Camden 020D',
       'Camden 026A', 'Camden 026B', 'Camden 026D', 'Camden 028A',
       'Camden 028B', 'Camden 028C', 'Camden 028D',
       'Hammersmith and Fulham 020E', 'Hammersmith and Fulham 024A',
       'Kensington and Chelsea 001A', 'Kensington and Chelsea 001C',
       'Kensington and Chelsea 001E', 'Kensington and Chelsea 004A',
       'Kensington and Chelsea 004B', 'Kensington and Chelsea 004D',
       'Kensington and Chelsea 006D', 'Kensington and Chelsea 006E',
       'Kensington and Chelsea 010E', 'Kensington and Chelsea 012A',
       'Kensington and Chelsea 012B', 'Kensington and Chelsea 012C',
       'Kensington and Chelsea 012E', 'Kensington and Chelsea 018C',
       'Kensington and Chelsea 018D', 'Southwark 024A']

noise_test_true = noise_test.loc[(noise_test.Service_Request_Group_Type == 'Noise Complaint')&(noise_test.LSOA_2011_Name  != '0')& (~noise_test.LSOA_2011_Name.isin(lsoanot) )]
noise_test_true

In [ ]:
# simple is the best
noise_test_true.Service_Request_Group_Sub_Type.unique()

In [ ]:
# "broken" (staggered) dates
dates = pd.Index([pd.Timestamp('2012-05-01'), 
                  pd.Timestamp('2012-05-04'), 
                  pd.Timestamp('2012-05-06')])
s = pd.Series([1, 2, 3], dates)

print(s.asfreq('D'))


In [ ]:
s1 = s.asfreq('D')
s1 

In [ ]:
s = noise_test_true.copy()
s = s.reset_index()
s

In [ ]:
# 
#s['2days_count'] = 0
#for i in range(len(s.Received_Date)):
#for i in range(100):
    #df = s.loc[ 
        #(
        #((s.days == s.days[i])  &  (s.Time > s.Time[i])) |
        #(s.days.isin(range(s.days[i]+1,s.days[i]+2))) 
        #) 
        #&
        #(  
        #s.Address_Key_ == s.Address_Key_[i] 
        #)
    #]
    #a = len(df.Noise_Complaint_Index)
    #s['2days_count'][i] = a

In [ ]:
#s['7days_count'] = 0
#for i in range(len(s.Received_Date)):
#for i in range(100):
    #df = s.loc[ 
        #(
        #((s.days == s.days[i])  &  (s.Time > s.Time[i])) |
        #(s.days.isin(range(s.days[i]+1,s.days[i]+7))) 
        #) 
        #&
        #(  
        #s.Address_Key_ == s.Address_Key_[i] 
        #)
    #]
    #a = len(df.Noise_Complaint_Index)
    #s['7days_count'][i] = a


In [ ]:
#s['30days_count'] = 0
#for i in range(len(s.Received_Date)):
##for i in range(100):
    #df = s.loc[ 
        #(
        #((s.days == s.days[i])  &  (s.Time > s.Time[i])) |
        #(s.days.isin(range(s.days[i]+1,s.days[i]+30))) 
        #) 
        #&
        #(  
        #s.Address_Key_ == s.Address_Key_[i] 
        #)
    #]
    #a = len(df.Noise_Complaint_Index)
    #s['30days_count'][i] = a

In [ ]:
s

In [ ]:
#s['bef_todays_count'] = 0
#for i in range(len(s.Received_Date)):
#for i in range(100):
    #df = s.loc[ 
        #(
        #((s.days == s.days[i])  &  (s.Time < s.Time[i])) 
        #) 
        #&
        #(  
        #s.Address_Key_ == s.Address_Key_[i] 
        #)
    #]
    #a = len(df.Noise_Complaint_Index)
    #s['bef_todays_count'][i] = a

In [ ]:
#s['bef_yesterday_count'] = 0
#for i in range(len(s.Received_Date)):
#for i in range(100):
    #df = s.loc[ 
        #(
        
        #(s.days.isin(range(s.days[i]-1,s.days[i])))
        #) 
        #&
        #(  
        #s.Address_Key_ == s.Address_Key_[i] 
        #)
    #]
    #a = len(df.Noise_Complaint_Index)
    #s['bef_yesterday_count'][i] = a

In [ ]:
#s['bef_2_7days_count'] = 0
#for i in range(len(s.Received_Date)):
#for i in range(100):
    #df = s.loc[ 
        #(
        
        #(s.days.isin(range(s.days[i]-7,s.days[i]-1)))
        #) 
        #&
        #(  
        #s.Address_Key_ == s.Address_Key_[i] 
        #)
    #]
    #a = len(df.Noise_Complaint_Index)
    #s['bef_2_7days_count'][i] = a

In [ ]:
#s['bef_7_30days_count'] = 0
#for i in range(len(s.Received_Date)):
#for i in range(100):
    #df = s.loc[ 
        #(
        
        #(s.days.isin(range(s.days[i]-30,s.days[i]-7)))
        #) 
        #&
        #(  
        #s.Address_Key_ == s.Address_Key_[i] 
        #)
    #]
    #a = len(df.Noise_Complaint_Index)
    #s['bef_7_30days_count'][i] = a

In [ ]:
#s['bef_30_365days_count'] = 0
#for i in range(len(s.Received_Date)):
#for i in range(100):
    #df = s.loc[ 
        #(
        #(s.days.isin(range(s.days[i]-365,s.days[i]-30)))
        #) 
       # &
        #(  
        #s.Address_Key_ == s.Address_Key_[i] 
        #)
    #]
    #a = len(df.Noise_Complaint_Index)
    #s['bef_30_365days_count'][i] = a

In [ ]:
#s['bef_365_begindays_count'] = 0
#for i in range(len(s.Received_Date)):
#for i in range(100):
   # df = s.loc[ 
        #(
        #(s.days.isin(range(0,s.days[i]-365)))
        #) 
        #&
        #(  
        #s.Address_Key_ == s.Address_Key_[i] 
        #)
    #]
    #a = len(df.Noise_Complaint_Index)
    #s['bef_365_begindays_count'][i] = a

In [ ]:
# 
s['2days_count'] = 0
for i in range(len(s.Received_Date)):
#for i in range(100):
    df = s.loc[ 
        (
        ((s.days == s.days[i])  &  (s.Time > s.Time[i])) |
        (s.days.isin(range(s.days[i]+1,s.days[i]+2))) 
        ) 
        &
        (  
        s.Address_Key_ == s.Address_Key_[i] 
        )
    ]
    a = len(df.Noise_Complaint_Index)
    s['2days_count'][i] = a

In [ ]:
s['7days_count'] = 0
for i in range(len(s.Received_Date)):
#for i in range(100):
    df = s.loc[ 
        (
        ((s.days == s.days[i])  &  (s.Time > s.Time[i])) |
        (s.days.isin(range(s.days[i]+1,s.days[i]+7))) 
        ) 
        &
        (  
        s.Address_Key_ == s.Address_Key_[i] 
        )
    ]
    a = len(df.Noise_Complaint_Index)
    s['7days_count'][i] = a


In [ ]:
s['30days_count'] = 0
for i in range(len(s.Received_Date)):
##for i in range(100):
    df = s.loc[ 
        (
        ((s.days == s.days[i])  &  (s.Time > s.Time[i])) |
        (s.days.isin(range(s.days[i]+1,s.days[i]+30))) 
        ) 
        &
        (  
        s.Address_Key_ == s.Address_Key_[i] 
        )
    ]
    a = len(df.Noise_Complaint_Index)
    s['30days_count'][i] = a

In [ ]:
s

In [ ]:
s['bef_todays_count'] = 0
for i in range(len(s.Received_Date)):
#for i in range(100):
    df = s.loc[ 
        (
        ((s.days == s.days[i])  &  (s.Time < s.Time[i])) 
        ) 
        &
        (  
        s.Address_Key_ == s.Address_Key_[i] 
        )
    ]
    a = len(df.Noise_Complaint_Index)
    s['bef_todays_count'][i] = a

In [ ]:
s['bef_yesterday_count'] = 0
for i in range(len(s.Received_Date)):
#for i in range(100):
    df = s.loc[ 
        (
        
        (s.days.isin(range(s.days[i]-1,s.days[i])))
        ) 
        &
        (  
        s.Address_Key_ == s.Address_Key_[i] 
        )
    ]
    a = len(df.Noise_Complaint_Index)
    s['bef_yesterday_count'][i] = a

In [ ]:
s['bef_2_7days_count'] = 0
for i in range(len(s.Received_Date)):
#for i in range(100):
    df = s.loc[ 
        (
        
        (s.days.isin(range(s.days[i]-7,s.days[i]-1)))
        ) 
        &
        (  
        s.Address_Key_ == s.Address_Key_[i] 
        )
    ]
    a = len(df.Noise_Complaint_Index)
    s['bef_2_7days_count'][i] = a

In [ ]:
s['bef_7_30days_count'] = 0
for i in range(len(s.Received_Date)):
#for i in range(100):
    df = s.loc[ 
        (
        
        (s.days.isin(range(s.days[i]-30,s.days[i]-7)))
        ) 
        &
        (  
        s.Address_Key_ == s.Address_Key_[i] 
        )
    ]
    a = len(df.Noise_Complaint_Index)
    s['bef_7_30days_count'][i] = a

In [ ]:
s['bef_30_365days_count'] = 0
for i in range(len(s.Received_Date)):
#for i in range(100):
    df = s.loc[ 
        (
        (s.days.isin(range(s.days[i]-365,s.days[i]-30)))
        ) 
        &
        (  
        s.Address_Key_ == s.Address_Key_[i] 
        )
    ]
    a = len(df.Noise_Complaint_Index)
    s['bef_30_365days_count'][i] = a

In [ ]:
s['bef_365_begindays_count'] = 0
for i in range(len(s.Received_Date)):
#for i in range(100):
    df = s.loc[ 
        (
        (s.days.isin(range(0,s.days[i]-365)))
        ) 
        &
        (  
        s.Address_Key_ == s.Address_Key_[i] 
        )
    ]
    a = len(df.Noise_Complaint_Index)
    s['bef_365_begindays_count'][i] = a

In [ ]:
#s.to_csv('3_future_count.csv')

In [ ]:
s = pd.read_csv('3_future_count.csv')
s

In [ ]:
s['is_repeat_2days'] = 0
s['is_repeat_7days'] = 0
s['is_repeat_30days'] = 0

s.loc[s['2days_count']>0, ['is_repeat_2days']] = 1
s.loc[s['7days_count']>0, ['is_repeat_7days']] = 1
s.loc[s['30days_count']>0, ['is_repeat_30days']] = 1

In [ ]:
#s['is_repeat_2days'] = 0

s['is_repeat_2_7days'] = 0
s['is_repeat_7_30days'] = 0

s.loc[(s['2days_count']==0)&(s['7days_count']>0), ['is_repeat_2_7days']] = 1
s.loc[(s['7days_count']==0)&(s['30days_count']>0), ['is_repeat_7_30days']] = 1



In [ ]:
s

In [ ]:

# The first 5% is 0 to 1315, count from 1620 to 9
#count_lsoa = noise_test_true.groupby(['LSOA_2011_Code']).count().sort_values(by=['Type_of_Address'],ascending=False).reset_index()
#lsoa_count= count_lsoa ['LSOA_2011_Code'][0:int(128*0.05)].unique()
#lsoa_count

In [ ]:
#count_lsoa = count_lsoa.rename(columns = {'Noise_Complaint_Index':'LSOA_Count'})
#count_lsoa

In [ ]:
#s['top5_lsoa_count'] = 0
#s.loc[s.LSOA_2011_Code.isin(lsoa_count),['top5_lsoa_count']] = 1
#s = s.merge(count_lsoa[['LSOA_2011_Code','LSOA_Count']], on ='LSOA_2011_Code' )

In [ ]:
s

In [ ]:
# The total count of address is 26303
# The first 5% is 0 to 1315, count from 1620 to 9
count_address = noise_test_true.groupby(['Address_Key_']).count().sort_values(by=['Type_of_Address'],ascending=False).reset_index()
#lsoa_count = count_address.loc[count_address.Noise_Complaint_Index>=50]['Address_Key_'].unique()
add_count= count_address['Address_Key_'][0:int(26303*0.05)].unique()
add_count


In [ ]:
count_address = count_address.rename(columns = {'Noise_Complaint_Index':'Adress_Count'})
count_address

In [ ]:
s['top5_address_count'] = 0
s.loc[s.Address_Key_.isin(add_count),['top5_address_count']] = 1
s = s.merge(count_address[['Address_Key_','Adress_Count']], on ='Address_Key_' )

In [ ]:
#  cluster

In [ ]:
lsoa_cluster_explained = pd.read_csv('2_explained_cluster.csv').drop(['Unnamed: 0'],axis =1)
cluster_dummies = pd.get_dummies(lsoa_cluster_explained['cluster'])
lsoa_cluster_explained = lsoa_cluster_explained.merge(cluster_dummies, left_index = True,right_index = True)
lsoa_cluster_explained = lsoa_cluster_explained.rename(columns = {0:'cluster0',1:'cluster1',2:'cluster2'})
lsoa_cluster_explained

In [ ]:
s_1 = s.merge(lsoa_cluster_explained[['LSOA11CD','cluster0','cluster1','cluster2']],left_on ='LSOA_2011_Code' ,right_on = 'LSOA11CD' )
#s_1 = s.merge(lsoa_cluster_explained,left_on ='LSOA_2011_Code' ,right_on = 'LSOA11CD' )

In [ ]:
s_1.columns

In [ ]:
s

In [ ]:
#high frequency hours

s_1['is_high_hour'] = 0
s_1.loc[s_1.Time.isin([20,21,22,23,0,1]), ['is_high_hour']] = 1

In [ ]:
#high frequency weekdays
s_1['is_high_weekday'] = 0
s_1.loc[s_1.Weekday.isin([5,6]), ['is_high_weekday']] = 1

In [ ]:
#high frequency month
s_1['is_high_month'] = 0
s_1.loc[s_1.Month.isin([6,7,8]), ['is_high_month']] = 1

In [ ]:
#lockdown

s_1['is_lock_down'] = 0
s_1.loc[s_1.Year_Month.isin(['2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10','2020-11','2020-12',
                            '2021-01','2021-02','2021-03','2021-04','2021-05','2021-06','2021-07']), ['is_lock_down']] = 1

In [ ]:
s_1

In [ ]:
#该投诉从属于哪一类
type_dummies = pd.get_dummies(s_1['Service_Request_Group_Sub_Type'])
s_2 = s_1.merge(type_dummies, left_index = True,right_index = True)
s_2

In [ ]:
s_2.columns

In [ ]:
s_2

In [ ]:
#s_2.to_csv('3_future_count_complete.csv')

In [ ]:
s_2 = pd.read_csv('3_future_count_complete.csv')

In [ ]:
s_2.columns